<a href="https://colab.research.google.com/github/sujin-create/hodlhodl/blob/Sentiment-analysis/Sentiment%20analysis/mysql_insert_ex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pymysql
!pip install matplotlib
!pip install seaborn


     |████████████████████████████████| 43 kB 856 kB/s 


In [ ]:
import pandas as pd 
import pymysql 
import matplotlib.pyplot as plt 
%matplotlib inline 
import seaborn as sns 
sns.set()

제공받은 csv파일을 이용한 mysql db저장 예제 코드

In [ ]:
crawling_data = pd.read_csv('2022-05-06_result.csv')

In [ ]:
crawling_data.head(10) #10개 뽑아보기 -> k값은 수정예정

,Unnamed: 0,date,comments,sentiment,K_weight
0,0,2022-05-06,If I go out and buy a $5 wrench and attack mys...,negative,-0.163366
1,1,2022-05-06,"Please remain seated, seatbelts on, we are exp...",neutral,-0.163366
2,2,2022-05-06,Didn’t sell a sat.,negative,-0.163366
3,3,2022-05-06,Holding and bought some extra on top of my dca 🤑,positive,-0.163366
4,4,2022-05-06,"I've been waiting for BTC to drop, yesterday l...",neutral,-0.163366
5,5,2022-05-06,"jobs report is positive, why we dumping",neutral,-0.163366
6,6,2022-05-06,"Not sure about anyone else, but I couldn't car...",negative,-0.163366
7,7,2022-05-06,If you just buy now and hold until at least th...,negative,-0.163366
8,8,2022-05-06,"Any other year, BTC would be at like $8k right...",neutral,-0.163366
9,9,2022-05-06,Unemployment report out in 15 mins. That’ll p...,negative,-0.163366


In [ ]:
# k값을 변경할 필요가있음 -> k값을 지금은 negative성질이 강한것으로 보이니 0.6정도로 임의로 설정
crawling_data.K_weight = 0.6
crawling_data

,Unnamed: 0,date,comments,sentiment,K_weight
0,0,2022-05-06,If I go out and buy a $5 wrench and attack mys...,negative,0.6
1,1,2022-05-06,"Please remain seated, seatbelts on, we are exp...",neutral,0.6
2,2,2022-05-06,Didn’t sell a sat.,negative,0.6
3,3,2022-05-06,Holding and bought some extra on top of my dca 🤑,positive,0.6
4,4,2022-05-06,"I've been waiting for BTC to drop, yesterday l...",neutral,0.6
...,...,...,...,...,...
197,197,2022-05-06,This daily thread can be so cringe lol,neutral,0.6
198,198,2022-05-06,This sub in a nutshell,positive,0.6
199,199,2022-05-06,>No it doesn't. I'm gonna buy more whatever th...,neutral,0.6
200,200,2022-05-06,I'm not sure to be honest. For a regular Bitco...,neutral,0.6


mysql에 데이터 insert 예제코드

In [ ]:
conn = pymysql.connect(host='autotrading-db.cjolqhecq70a.ap-northeast-2.rds.amazonaws.com',
                       user = 'admin',
                       password='dbpassword입력',
                       db = 'AutoTrading',
                       charset = 'utf8')
curs = conn.cursor() 

**1. sentiment table에 insert. 크롤링 날짜 및 댓글 그리고 감정분석 결과를 넣는다.**


In [ ]:
#저장할 date를 변수에 저장.
print(crawling_data['date'][0])
date = crawling_data['date'][0]
print(type(date))

#db에 알아보기 쉽도록 count값을 저장. -> count값을 계산하는 예제코드
print(len(crawling_data))

n_count_filter = (crawling_data.sentiment == 'negative')
print(len(crawling_data[n_count_filter]))
n_count = len(crawling_data[n_count_filter])

p_count_filter = (crawling_data.sentiment == 'positive')
print(len(crawling_data[p_count_filter]))
p_count = len(crawling_data[p_count_filter])

neu_count_filter = (crawling_data.sentiment == 'neutral')
print(len(crawling_data[neu_count_filter]))
neu_count = len(crawling_data[neu_count_filter])

print(crawling_data.K_weight[0])
k_val = crawling_data.K_weight[0]


2022-05-06
<class 'str'>
202
82
49
71
0.6


In [ ]:
#하루마다 sentiment table의 row가 1개씩 증가.
insert_sentiment = """insert into sentiment(crawling_date,n_count,p_count,neu_count,k_val)
         values (%s, %s, %s, %s, %s)"""
curs.execute(insert_sentiment, (date, n_count,p_count, neu_count, k_val))
conn.commit()

conn.close()

**2. comments table에 insert.댓글과 각 댓글의 감정분석 결과 그리고 그 들의 크롤링 날짜를 넣는다.**

In [ ]:
conn = pymysql.connect(host='autotrading-db.cjolqhecq70a.ap-northeast-2.rds.amazonaws.com',
                       user = 'admin',
                       password='dbpassword입력',
                       db = 'AutoTrading',
                       charset = 'utf8mb4')
curs = conn.cursor() 

In [ ]:
# 저장할 date를 변수에 저장 -> 위의 date와 일치.

sql_rows = []

for i in range(0,len(crawling_data)):
    sql_row = (date,crawling_data['comments'][i],crawling_data['sentiment'][i])
    sql_rows.append(sql_row)


print(sql_rows)



[('2022-05-06', "If I go out and buy a $5 wrench and attack myself with it, how long do you think it will take until I'm immune?", 'negative'), ('2022-05-06', 'Please remain seated, seatbelts on, we are experiencing some slight turbulence. In the meantime please peruse the inflight magazine were you’ll find some excellent offers, as fiat crashes violently into the sea.', 'neutral'), ('2022-05-06', 'Didn’t sell a sat.', 'negative'), ('2022-05-06', 'Holding and bought some extra on top of my dca 🤑', 'positive'), ('2022-05-06', "I've been waiting for BTC to drop, yesterday loaded my first one.", 'neutral'), ('2022-05-06', 'jobs report is positive, why we dumping', 'neutral'), ('2022-05-06', "Not sure about anyone else, but I couldn't care less about the current market conditions.   In the long run this is all just noise.  It's silly that people are upset.", 'negative'), ('2022-05-06', 'If you just buy now and hold until at least the year after the next halving, any inflation hike on whate

In [ ]:
insert_comment = """insert into comments(crawling_date,comment,sentiment) values (%s, %s, %s)"""
curs.executemany(insert_comment,sql_rows)
conn.commit()
conn.close()
